In [ ]:
import numpy as np 
import math


def safe_softmax(vector): 
  l = np.size(vector)
  out = np.zeros(l)
  maxim = float("-inf")
  denom = 0 
  #first pass, calculate max 
  for i in range(l): 
    maxim = max(vector[i],maxim)
  #second pass, calculate denom 
  for i in range(l): 
    denom += math.exp(vector[i] - maxim)
  #third pass, calculate softmax 
  for i in range(l):
    out[i] = math.exp(vector[i]-maxim)/denom
    
  return out 

def ref_softmax(vector): 
  max = np.max(vector)
  denom = np.sum(np.exp(vector-max))
  return np.exp(vector-max)/denom


def safe_online_softmax(vector): 
  prev_maxim = float("-inf")
  prev_denom = 0
  maxim = prev_maxim
  denom = prev_denom
  l = np.size(vector)
  out = np.zeros(l)
  for i in range(l): 
    maxim = max(prev_maxim, vector[i])
    denom = prev_denom*math.exp(prev_maxim-maxim) + math.exp(vector[i]-maxim)
    prev_maxim = maxim 
    prev_denom = denom 
    
  for i in range(l): 
    out[i] = math.exp(vector[i]-maxim)/denom
  return out



In [35]:
vector = np.random.randn(16)
ref_out = ref_softmax(vector)
out = safe_softmax(vector)
online_out = safe_online_softmax(vector)


In [36]:
online_out


array([0.00977199, 0.04291437, 0.03865582, 0.24593633, 0.07027081,
       0.0049527 , 0.05265564, 0.1021273 , 0.01248684, 0.1194562 ,
       0.04518149, 0.01826486, 0.04192244, 0.0870057 , 0.08464499,
       0.02375251])

In [37]:
ref_out

array([0.00977199, 0.04291437, 0.03865582, 0.24593633, 0.07027081,
       0.0049527 , 0.05265564, 0.1021273 , 0.01248684, 0.1194562 ,
       0.04518149, 0.01826486, 0.04192244, 0.0870057 , 0.08464499,
       0.02375251])

In [33]:
x = np.array([float('-inf')]*10)

array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf])